In [1]:
import sys
import os

# Then proceed to import and use Spark
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("CricketPrediction").getOrCreate()


# Specify the directory where your CSV files are located
directory = r'D:\github\Cricket-Prediction\data\2_processeddata'

# Read batting,bowling,fielding CSV files from the specified directory
batting = spark.read.csv(directory + r'\batting.csv', header=True, inferSchema=True)
bowling = spark.read.csv(directory + r'\bowling.csv', header=True, inferSchema=True).drop('Mat','Inns')
fielding = spark.read.csv(directory + r'\fielding.csv', header=True, inferSchema=True).drop('Mat','Inns')
batting.show(5)

KeyboardInterrupt: 

In [ ]:
# Check for nulls in the data
def check_nulls(df):
    nulls = 0
    for col in df.columns:
        nulls+= df.filter(df[col].isNull()).count()
    print("Total nulls in the dataframe: ", nulls)

check_nulls(batting)
check_nulls(bowling)
check_nulls(fielding)

Total nulls in the dataframe:  0
Total nulls in the dataframe:  0
Total nulls in the dataframe:  0


In [ ]:
playerdata = batting.join(bowling, on=['player_id','Player',"Country","Season"], how='inner').join(fielding, on=['player_id','Player',"Country","Season"], how='inner')\
    .drop('Cumulative Mat','Cumulative Inns')
playerdata = playerdata.dropDuplicates(['player_id', 'Player', 'Country', 'Season'])
playerdata.show(5)

+---------+----------+--------+-------+-------------+--------------+--------------+---------------+------+------------------+---------------+---------------+---------------+--------------+-------------+-------------+--------------+
|player_id|    Player| Country| Season|Cum Mat Total|Cum Inns Total|Cum Runs Total|Cum Batting Ave|Cum SR|  Cumulative Overs|Cumulative Runs|Cumulative Wkts|Cumulative Econ|Cumulative Dis|Cumulative Ct|Cumulative St|Cumulative D/I|
+---------+----------+--------+-------+-------------+--------------+--------------+---------------+------+------------------+---------------+---------------+---------------+--------------+-------------+-------------+--------------+
| 55a5cffb|A Ahmadhel|Bulgaria|2019/20|            0|             0|             0|            0.0|   0.0|               0.0|            0.0|            0.0|            0.0|           0.0|          0.0|          0.0|           0.0|
| 55a5cffb|A Ahmadhel|Bulgaria|   2020|            3|             2|    

In [ ]:
directory = r'D:\github\Cricket-Prediction\data\2_processeddata'
playerdata.toPandas().to_csv(directory + r'\playerStats.csv', index=False)
spark.stop()